In [15]:
import openai
import os


def geo_classify(title, text):
  
    prompt = f'''
    Please extract all specifc geographic locations or pysical places or buildings, return them as a comma separated list.

    Do not include casual decriptions eg. "behind the church". 

    Title: {title}

    Text: {text}

    '''


    # Set your OpenAI API key
    api_key = os.environ["OPENAI_API_KEY"]

    # Initialize the OpenAI API client
    openai.api_key = api_key

    # Define the message you want to send
    message = prompt



    # Send the message to ChatGPT-4 and get a response
    completion = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": message,
            },
        ],
    )
    return completion.choices[0].message.content

In [22]:
from postgres import Postgres 
import json

pg = Postgres()

records = pg.query("""
    SELECT id, cmetadata->>'neighbourhood' as neighbourhood, cmetadata->>'block_idx' as block_idx, cmetadata->>'sections' as sections, cmetadata->>'text' as text
    FROM langchain_pg_embedding
    WHERE NOT (cmetadata ? 'openai_geo_labels') 
    AND cmetadata->>'experiment' = 'leeds'
    AND cmetadata->>'chunker' = 'sherpa'
    
    LIMIT 1000;
""")
print('records: ', records)

for record in records:
    title = record[3]
    text = record[4]
    
    comma_separated_labels = geo_classify(title, text)

    labels = [item.strip() for item in comma_separated_labels.split(",")]       
    
    print ('labels: ', labels)
    id = record[0]
    title = record[3]
    text = record[4]
    print('title: ', title)
    print('text: ', text)
    
    print ('labels: ', labels)
    pg.add_labels_to_cmetadata(id, labels, 'openai_geo_labels')
    

    

records:  [('59d135e2-5ad4-43a5-aa13-3f1f5aae574c', 'Clifford', '30', 'introduction', ' Once this Neighbourhood Plan is ‘made’ as part of the statutory development plan, it will be a primary consideration when determining planning applications within the Plan area.   The Neighbourhood Plan has been prepared such that Clifford parish stakeholders (residents, businesses, landowners and other organisations) can have a real influence over what development takes place in the parish and ensuring that its rural and historic character are protected - while at the same time ensuring that it remains a vibrant and sustainable settlement.\nThe development of the Neighbourhood Plan is based on extensive research and is influenced by extensive engagement with these stakeholders.\nThe policies and proposals set out the clear wishes of the stakeholders in the parish. '), ('b471a4c8-1200-415b-a6a2-24331d38aaa0', 'Barwick in Elmet & Scholes', '27', 'Countryside character > The countryside around Scholes